In [29]:
import pandas as pd
import numpy as np 
import math

In [30]:
# number
def add(a:int|float, b:int|float) ->int|float:
    return a+b
def subtract(a:int|float, b:int|float) ->int|float:
    return a-b
def multiply(a:int|float, b:int|float) ->int|float:
    return a*b
def divide(a:int|float, b:int|float) ->float:
    return a/b
def power_of(a:int|float, of:int = 2)->int|float:
    return a**of
def root_of(a:int|float, base:int = 2)->int|float:
    return a**(1/base)

In [31]:
# int
def int_div(a:int, b:int)->int:
    return a//b

def modulo(a:int, b:int)->int:
    a%b

In [32]:
#float
def _round(a:float, places:int = 2) ->float:
    return round(a,places)

def floor(a:float)->int:
    return math.floor(a)

def ceil(a:float)->int:
    return math.ceil(a)


In [33]:
#string 
def concat(a:str, b:str)->str:
    return a+b

def substr(a:str, start:int, end:int)->str:
    return a[start:end]

def strip(a:str, special_char:str|None = None)->str:
    return a.strip(special_char)

def split(a:str, on_what:str, keep:int = 0)->str:
    return a.split(on_what)[keep]

def replace(a:str, where:int, _with:str)->str:
    return a[:where]+_with+a[where+1:]

def replace_all(a:str, what:str, _whith:str)->str:
    return a.replace(what, _whith)


In [34]:
#bool 
def _not(b:bool)->bool:
    return not b

def _and(a:bool, b:bool)->bool:
    return a and b

def _or(a:bool, b:bool)->bool:
    return a or b

def _xor(a:bool, b:bool)->bool:
    return a^b

In [35]:
#data conversion
def toString(a:int|float)->str:
    return str(a)

def toInt(a:str|float)->int:
    return int(a)

def toFloat(a:str|int)->float:
    float(a)

In [36]:
# data reduction 
def _sum(a)->int|float:
    return np.sum(a)

def _prod(a)->int|float:
    return np.prod(a)

def _min(a)->int|float:
    return np.min(a)

def _max(a)->int|float:
    return np.max(a)

def _mean(a)->int|float:
    return np.mean(a)

In [37]:
# data evaluation 
def is_empty(a)->bool:
    return pd.isnull(a)

def contains(a:str, c_str:str, at:int|None = None)->bool:
    if at is not None:
        return a[at] == c_str
    return c_str in a

def starts_with(a:str, c_str:str)->str:
    a.startswith(c_str)

In [ ]:
expr_locals = {
    
}

### testing

In [42]:
df = pd.read_excel("alldtypes.xlsx")
df = df.replace({np.nan:None})
df

,INT,FLOAT,STRING,NONE,BOOL
0,1,1.1,text,None,True
1,2,2.1,text mit Spaces,None,False
2,3,3.3,"sumbole,%§/(",None,True


str